In [1]:
# install depedencies
!pip install pydmd

In [8]:
# import all dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pydmd import DMD

In [9]:
# import files from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# load data
series = pd.read_csv('/content/drive/My Drive/Dynamic-Mode-Decomposition/airline_passengers.csv', index_col=0, header=0, parse_dates=True, squeeze=True)
print(series)

Month
1949-01-01    112
1949-02-01    118
1949-03-01    132
1949-04-01    129
1949-05-01    121
             ... 
1960-08-01    606
1960-09-01    508
1960-10-01    461
1960-11-01    390
1960-12-01    432
Name: Passengers, Length: 144, dtype: int64


In [12]:
# plot data
series.plot(figsize=(12, 5))
plt.show()

In [13]:
# split data into a training and validation datasets
split_point = len(series) - 12
training, validation = series[0:split_point], series[split_point:]
print(f'Training {len(training)}, Validation {len(validation)}')
training.to_csv('/content/drive/My Drive/Dynamic-Mode-Decomposition/airline_passengers_training.csv', header=False)
validation.to_csv('/content/drive/My Drive/Dynamic-Mode-Decomposition/airline_passengers_validation.csv', header=False)

Training 132, Validation 12


In [14]:
# load training dataset
training = pd.read_csv('/content/drive/My Drive/Dynamic-Mode-Decomposition/airline_passengers_training.csv', index_col=0, header=None, parse_dates=True, squeeze=True)
print(training.values)

[112 118 132 129 121 135 148 148 136 119 104 118 115 126 141 135 125 149
 170 170 158 133 114 140 145 150 178 163 172 178 199 199 184 162 146 166
 171 180 193 181 183 218 230 242 209 191 172 194 196 196 236 235 229 243
 264 272 237 211 180 201 204 188 235 227 234 264 302 293 259 229 203 229
 242 233 267 269 270 315 364 347 312 274 237 278 284 277 317 313 318 374
 413 405 355 306 271 306 315 301 356 348 355 422 465 467 404 347 305 336
 340 318 362 348 363 435 491 505 404 359 310 337 360 342 406 396 420 472
 548 559 463 407 362 405]


In [15]:
# create and fit a DMD model
dmd = DMD(svd_rank=-1,
          opt=False,
          exact=False,
          rescale_mode='auto',
          forward_backward=True,
          tlsq_rank=1)
dmd.fit(training.values)

In [16]:
x = training.values
#print(type(x))
#print(x)
x = x.reshape((132,1))
#print(x.shape)
#print(x)
x_test = x[131].reshape((1,1))
#print(x_test.shape)
yhat = dmd.predict(x_test)
print(x[131])
print(yhat)
print(dmd.reconstructed_data.real[0])

[405]
[[407.95117607]]
[112.         112.8161277  113.63820241 114.46626746 115.30036649
 116.14054349 116.98684273 117.83930883 118.69798673 119.56292168
 120.4341593  121.31174549 122.19572653 123.08614901 123.98305987
 124.88650639 125.79653619 126.71319724 127.63653787 128.56660675
 129.50345291 130.44712573 131.39767495 132.35515069 133.31960342
 134.29108397 135.26964356 136.25533377 137.24820656 138.24831427
 139.25570962 140.27044571 141.29257604 142.32215448 143.35923532
 144.4038732  145.45612322 146.51604082 147.58368189 148.65910271
 149.74235996 150.83351075 151.93261259 153.03972344 154.15490163
 155.27820597 156.40969566 157.54943036 158.69747013 159.85387551
 161.01870744 162.19202733 163.37389703 164.56437885 165.76353553
 166.9714303  168.18812682 169.41368922 170.64818213 171.8916706
 173.14422019 174.40589693 175.67676733 176.95689837 178.24635754
 179.54521281 180.85353265 182.17138603 183.49884241 184.83597178
 186.18284461 187.53953191 188.90610519 190.28263649 1

In [17]:
x = training.values
x = x.reshape((132,1))
forecasts = []
for i in range(len(x)):
    x_i = x[i].reshape((1,1))
    yhat = dmd.predict(x_i)
    forecasts.append(yhat)
forecasts = np.array(forecasts).ravel()

In [18]:
train_idx = np.linspace(0, len(training), len(training))
#print(len(train_idx))
#print(training.values)
#print(dmd.original_timesteps)
#print(dmd.dmd_timesteps)
#print(len(dmd.reconstructed_data.real[0]))

In [19]:
# plot predictions on training data
#train_idx = np.linspace(0, len(training), len(training))
#dmd.original_time['dt'] = dmd.dmd_time['dt'] = train_idx[1] - train_idx[0]
#dmd.original_time['t0'] = dmd.dmd_time['t0'] = train_idx[0]
#dmd.original_time['tend'] = dmd.dmd_time['tend'] = train_idx[-1]
plt.plot(training.values, label='Actuals')
plt.plot(forecasts, label='Predictions')
plt.legend()
plt.show()